#### Generates Weight Sheets

In [1]:
import pandas as pd
import pdfkit
import json
import base64
import os

# Names for the output PDFs
tournament_code = "almondaCup_2024"

# Read the Excel file without skipping rows
df = pd.read_excel('Input/240413 INSC_PROVA III ALMONDA CUP.xlsx', header=None)

# Drop the first row ( TITLE )
df = df.drop(index=[0,1])

# Dropping unnecessary columns
columns_to_drop = [0, 1, 2, 5, 7, 8, 9]
df = df.drop(columns=columns_to_drop)

# Reorder the columns
df = df[[4, 10, 6, 3]]

# Set new column names
df.columns = ['NAME', 'WGT_CAT', 'GENDER', 'CLUB']

# Sort the DataFrame by the 'CLUB' column
df = df.sort_values(by=['CLUB', 'WGT_CAT'])

In [2]:
# Create separate DataFrames for masculine and feminine athletes
masculine_df = df[df['GENDER'] == 'MASC']
feminine_df = df[df['GENDER'] == 'FEM']

# Create a dictionary to store athletes for each club
def create_club_dict(athletes):
    club_athletes = {}
    for club in athletes['CLUB'].unique():
        club_athletes[club] = athletes[athletes['CLUB'] == club]
    return club_athletes

masculine_club_athletes = create_club_dict(masculine_df)
feminine_club_athletes = create_club_dict(feminine_df)

total_masculine_athletes = sum(len(athletes) for athletes in masculine_club_athletes.values())
total_feminine_athletes = sum(len(athletes) for athletes in feminine_club_athletes.values())
total_athletes = total_masculine_athletes + total_feminine_athletes

print(f"""
Inscritos:
      Masculinos:   {total_masculine_athletes}
      Femininos:    {total_feminine_athletes}
      Total:        {total_athletes}
""")


Inscritos:
      Masculinos:   111
      Femininos:    44
      Total:        155



In [3]:
# Load the JSON data
with open('clubs.json') as f:
    clubs_data = json.load(f)

# Create a dictionary from the JSON data
clubs_dict = {club['initials']: {'full_name': club['full_name'], 'image': club['image']} for club in clubs_data['clubs']}

In [4]:
# Define the HTML template for club and athlete information
template = """
<!DOCTYPE html>
<html>
<head>
    <style>
        table {{
            border-collapse: collapse;
            width: 100%;
            margin-bottom: 50px;
            page-break-inside: avoid;
        }}
        th, td {{
            border: 1px solid black;
            text-align: center;
        }}
        th {{
            background-color: #f2f2f2;
            font-size: 15px;
        }}
        .td_name {{
            //text-align: left;
            font-size: 13px;
        }}
        .width_fix {{
            width: 90px;
        }}
        img {{
            max-width: 100px;
            max-height: 100px;
        }}
        .checkbox {{
            font-size: 24px;
        }}
    </style>
</head>
<body>
    <table>
        <tr>
            <th style="width: 100px; height: 100px">{emblem}</th>
            <th colspan="8" style="text-align: left; padding-left: 12px; font-size: 19px;">{fullname}</th>
        </tr>
        <tr>
            <th>Nome do Atleta</th>
            <th style="width: 120px">Categoria de Inscrição</th>
            <th class="width_fix">Presente</th>
            <th class="width_fix">Peso Atual</th>
            <th class="width_fix">Mantém Categoria</th>
            <th class="width_fix">Sobe Categoria</th>
            <th class="width_fix">Desce Categoria</th>
            <th class="width_fix">Nova Categoria</th>
        </tr>
        {rows}
    </table>
</body>
</html>
"""

```shell
sudo chmod 0700 /run/user/1000/
```

In [5]:
# Function to generate PDF for each club
def generate_club_pdf(club_athletes, filename):
    
    options = {
        'encoding': "UTF-8",
        'page-size': 'A4',
        'orientation': 'Portrait',
        'margin-top': '0.75in',
        'margin-right': '0.5in',
        'margin-bottom': '0.75in',
        'margin-left': '0.5in',
    }

    pdf_content = []
    for club, athletes in club_athletes.items():
        # Check if the club exists in the clubs_dict using its full name
        club_info = None
        for info in clubs_dict.values():
            if info['full_name'].upper() == club.upper():
                club_info = info
                break

        # If the club is not found using its full name, fallback to using initials
        if not club_info:
            club_info = clubs_dict.get(club)

        # Use the club information if found
        if club_info:
            club_name = club_info['full_name']
            if club_info['image'] is not None and club_info['image'] != '':
                club_image_path = club_info['image']
            else:
                club_image_path = "imgClubs/placeholder.jpg"
        else:
            club_name = club  # Use initials if full name not found
            if club_info['image'] is not None and club_info['image'] != '':
                club_image_path = club_info['image']
            else:
                club_image_path = "imgClubs/placeholder.jpg"

        # Convert image to base64 encoding
        img_data = ''
        if club_image_path is not None:
            try:
                with open(club_image_path, 'rb') as f:
                    img_data = base64.b64encode(f.read()).decode('utf-8')
                    img_tag = f'<img src="data:image/jpeg;base64,{img_data}">' if img_data else ''
                    emblem = f"{img_tag}"
            except FileNotFoundError:
                # Handle the case where the image file is not found
                print(f"Warning: Image file '{club_image_path}' not found for club '{club_name}'")
                club_image_path = None  # Set club_image_path to None to prevent further processing

        fullname = f"{club_name}"

        # Generate HTML for club and athlete information
        rows_html = ""
        for _, row in athletes.iterrows():
            rows_html += f"<tr> <td class=\"td_name\">{row['NAME']}</td> <td>{row['WGT_CAT']}</td> <td class=\"checkbox\">&#9744;</td> <td>_____</td> <td class=\"checkbox\">&#9744;</td> <td class=\"checkbox\">&#9744;</td> <td class=\"checkbox\">&#9744;</td> <td>_____</td></tr>"
        html_content = template.format(emblem = emblem, fullname= fullname, rows=rows_html)
        pdf_content.append(html_content)

    # Convert HTML to PDF
    pdfkit.from_string(''.join(pdf_content), filename, options=options)


if not os.path.exists("Output"):
    os.makedirs("Output")
    print("Output folder does not exist. Generating...")

# Generate PDF for masculine athletes
generate_club_pdf(masculine_club_athletes, f'Output/{tournament_code}_masculinos.pdf')

# Generate PDF for feminine athletes
generate_club_pdf(feminine_club_athletes, f'Output/{tournament_code}_femininos.pdf')